## NYPD Dataset

Dataset description at 
https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i



| Column | Description |
|--------|-------------------|
| CMPLNT_NUM |  Randomly generated persistent ID for each complaint  |  
| ADDR_PCT_CD |  The precinct in which the incident occurred |  
| BORO |  The name of the borough in which the incident occurred |  
| CMPLNT_FR_DT |  Exact date of occurrence for the reported event (or starting date of occurrence, if CMPLNT_TO_DT exists) |  
| CMPLNT_FR_TM |  Exact time of occurrence for the reported event (or starting time of occurrence, if CMPLNT_TO_TM exists) |  
| CMPLNT_TO_DT |  Ending date of occurrence for the reported event, if exact time of occurrence is unknown |  
| CMPLNT_TO_TM |  Ending time of occurrence for the reported event, if exact time of occurrence is unknown |  
| CRM_ATPT_CPTD_CD |  Indicator of whether crime was successfully completed or attempted, but failed or was interrupted prematurely |  
| HADEVELOPT |  Name of NYCHA housing development of occurrence, if applicable |  
| HOUSING_PSA |  Development Level Code |  
| JURISDICTION_CODE |  Jurisdiction responsible for incident. Either internal, like Police(0), Transit(1), and Housing(2); or external(3), like Correction, Port Authority, etc. |  
| JURIS_DESC |  Description of the jurisdiction code |  
| KY_CD |  Three digit offense classification code |  
| LAW_CAT_CD |  Level of offense: felony, misdemeanor, violation  |  
| LOC_OF_OCCUR_DESC |  Specific location of occurrence in or around the premises; inside, opposite of, front of, rear of |  
| OFNS_DESC |  Description of offense corresponding with key code |  
| PARKS_NM |  Name of NYC park, playground or greenspace of occurrence, if applicable (state parks are not included) |  
| PATROL_BORO |  The name of the patrol borough in which the incident occurred |  
| PD_CD |  Three digit internal classification code (more granular than Key Code) |  
| PD_DESC |  Description of internal classification corresponding with PD code (more granular than Offense Description) |  
| PREM_TYP_DESC |  Specific description of premises; grocery store, residence, street, etc. |  
| RPT_DT |  Date event was reported to police  |  
| STATION_NAME |  Transit station name |  
| SUSP_AGE_GROUP |  Suspect’s Age Group |  
| SUSP_RACE |  Suspect’s Race Description |  
| SUSP_SEX |  Suspect’s Sex Description |  
| TRANSIT_DISTRICT |  Transit district in which the offense occurred. |  
| VIC_AGE_GROUP |  Victim’s Age Group |  
| VIC_RACE |  Victim’s Race Description |  
| VIC_SEX |  Victim’s Sex Description |  
| X_COORD_CD |  X-coordinate for New York State Plane Coordinate System, Long Island Zone, NAD 83, units feet (FIPS 3104) |  
| Y_COORD_CD |  Y-coordinate for New York State Plane Coordinate System, Long Island Zone, NAD 83, units feet (FIPS 3104) |  
| Latitude |  Midblock Latitude coordinate for Global Coordinate System, WGS 1984, decimal degrees (EPSG 4326)  |  
| Longitude |  Midblock Longitude coordinate for Global Coordinate System, WGS 1984, decimal degrees (EPSG 4326) |


In [1]:
import pandas as pd
import numpy as np

In [2]:
# We load everything as an object/string, because some data types (e.g., some IDs)
# are recognized as decimals, and it is a mess to restore them back
# So we will do all the conversions ourselves later on

# From https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i/data
!curl 'https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD' -o nypd.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2243M    0 2243M    0     0  4508k      0 --:--:--  0:08:29 --:--:-- 5123k


In [3]:
%%time
df = pd.read_csv('nypd.csv', low_memory = True, dtype='object')

CPU times: user 40.3 s, sys: 6.25 s, total: 46.5 s
Wall time: 46.3 s


In [4]:
# We load directly from the URL
# url = 'https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD'
# df = pd.read_csv(url, low_memory = True, dtype='object')


In [5]:
len(df)

7375993

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7375993 entries, 0 to 7375992
Data columns (total 35 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   CMPLNT_NUM         object
 1   CMPLNT_FR_DT       object
 2   CMPLNT_FR_TM       object
 3   CMPLNT_TO_DT       object
 4   CMPLNT_TO_TM       object
 5   ADDR_PCT_CD        object
 6   RPT_DT             object
 7   KY_CD              object
 8   OFNS_DESC          object
 9   PD_CD              object
 10  PD_DESC            object
 11  CRM_ATPT_CPTD_CD   object
 12  LAW_CAT_CD         object
 13  BORO_NM            object
 14  LOC_OF_OCCUR_DESC  object
 15  PREM_TYP_DESC      object
 16  JURIS_DESC         object
 17  JURISDICTION_CODE  object
 18  PARKS_NM           object
 19  HADEVELOPT         object
 20  HOUSING_PSA        object
 21  X_COORD_CD         object
 22  Y_COORD_CD         object
 23  SUSP_AGE_GROUP     object
 24  SUSP_RACE          object
 25  SUSP_SEX           object
 26  TRANSIT_DISTRI

## Data Cleaning

In [7]:
# These columns are redundant
to_drop = ['Lat_Lon','X_COORD_CD','Y_COORD_CD']
# We created the CMPLNT_FR and CMPLNT_TO columns
# We have the longitude and latitude so the other coordinates are not needed
df = df.drop(to_drop, axis='columns')

###  CMPLNT_NUM         object   

In [8]:
     
# Drop cases with duplicated complaint numbers
key_cnt = df.CMPLNT_NUM.value_counts()
key_cnt [ key_cnt>1 ]
df = df[ ~df.CMPLNT_NUM.isin( key_cnt [ key_cnt>1 ].index.values ) ]

In [9]:
df.CMPLNT_NUM = df.CMPLNT_NUM.astype('int32')

### CMPLNT_FR_DT       object
### CMPLNT_FR_TM       object
### CMPLNT_TO_DT       object
### CMPLNT_TO_TM       object

In [10]:
# There are a few rows that contain year 1015, 1016, ... that trigger an error during date conversion
# We replace all years written as 10XX with 20XX
# Note the usage of regular expressions
df.CMPLNT_FR_DT.replace(to_replace = '(\d\d)/(\d\d)/10(\d\d)', value=r'\1/\2/20\3', inplace = True, regex=True )
df.CMPLNT_TO_DT.replace(to_replace = '(\d\d)/(\d\d)/10(\d\d)', value=r'\1/\2/20\3', inplace = True, regex=True )

# Similarly, a few hours are written as 24:00:00, which also triggers errors.
# We fix these hours
df.CMPLNT_FR_TM.replace(to_replace = '24:00:00', value='00:00:00', inplace = True)
df.CMPLNT_TO_TM.replace(to_replace = '24:00:00', value='00:00:00', inplace = True)

# Convert the two separate date and time columns into single datetime columns
df['CMPLNT_FR'] = pd.to_datetime(df.CMPLNT_FR_DT + ' ' + df.CMPLNT_FR_TM, format='%m/%d/%Y %H:%M:%S', cache=True)
df['CMPLNT_TO'] = pd.to_datetime(df.CMPLNT_TO_DT + ' ' + df.CMPLNT_TO_TM, format='%m/%d/%Y %H:%M:%S', cache=True)

# These columns are redundant
to_drop = ['CMPLNT_FR_DT','CMPLNT_TO_DT','CMPLNT_FR_TM','CMPLNT_TO_TM']
# We created the CMPLNT_FR and CMPLNT_TO columns
# We have the longitude and latitude so the other coordinates are not needed
df = df.drop(to_drop, axis='columns')

In [11]:
df.CMPLNT_FR.isnull().sum()

702

In [12]:
df.CMPLNT_TO.isnull().sum()

1704342

In [13]:
df = df [ ~df.CMPLNT_FR.isnull() ]

###  ADDR_PCT_CD        object

In [14]:

df.ADDR_PCT_CD.replace(to_replace = '-99', value='99', inplace = True)
# df = df [ ~df.ADDR_PCT_CD.isnull() ]
df.ADDR_PCT_CD = pd.Categorical(df.ADDR_PCT_CD)

###  RPT_DT             object

In [15]:


# Convert RPT_DT to date
df.RPT_DT = pd.to_datetime(df.RPT_DT, format="%m/%d/%Y", cache=True)

### 7   KY_CD              object
### 8   OFNS_DESC          object

In [16]:
df.OFNS_DESC.replace(to_replace = 'KIDNAPPING', value='KIDNAPPING & RELATED OFFENSES', inplace = True)
df.OFNS_DESC.replace(to_replace = 'KIDNAPPING AND RELATED OFFENSES', value='KIDNAPPING & RELATED OFFENSES', inplace = True)

df.OFNS_DESC.replace(to_replace = 'AGRICULTURE & MRKTS LAW-UNCLASSIFIED', value='OTHER STATE LAWS (NON PENAL LAW)', inplace = True)
df.OFNS_DESC.replace(to_replace = 'OTHER STATE LAWS (NON PENAL LA', value='OTHER STATE LAWS (NON PENAL LAW)', inplace = True)

df.OFNS_DESC.replace(to_replace = 'ENDAN WELFARE INCOMP', value='OFFENSES RELATED TO CHILDREN', inplace = True)

df.OFNS_DESC.replace(to_replace = 'THEFT OF SERVICES', value='OTHER OFFENSES RELATED TO THEF', inplace = True)

df.OFNS_DESC.replace(to_replace = 'NYS LAWS-UNCLASSIFIED VIOLATION', value='OTHER STATE LAWS', inplace = True)

df.OFNS_DESC.replace(to_replace = 'FELONY SEX CRIMES', value='SEX CRIMES', inplace = True)

df.loc[df.KY_CD=='120','OFNS_DESC'] ='CHILD ABANDONMENT/NON SUPPORT'

df.loc[df.KY_CD=='125','OFNS_DESC'] ='NYS LAWS-UNCLASSIFIED FELONY'

offenses = df[ ["KY_CD", "OFNS_DESC"] ].drop_duplicates().dropna()
offenses['KY_CD'] = pd.Categorical(pd.to_numeric(offenses['KY_CD'] ).astype(int))
offenses = offenses.set_index("KY_CD")
offenses = offenses.sort_index()
offenses = offenses.reset_index()
offenses


,KY_CD,OFNS_DESC
0,101,MURDER & NON-NEGL. MANSLAUGHTER
1,102,HOMICIDE-NEGLIGENT-VEHICLE
2,103,"HOMICIDE-NEGLIGENT,UNCLASSIFIE"
3,104,RAPE
4,105,ROBBERY
...,...,...
69,676,NEW YORK CITY HEALTH CODE
70,677,OTHER STATE LAWS
71,678,MISCELLANEOUS PENAL LAW
72,685,ADMINISTRATIVE CODES


In [17]:
df.KY_CD = pd.Categorical(df.KY_CD)

In [18]:
df = df.drop('OFNS_DESC', axis='columns')

### 9   PD_CD              object
### 10  PD_DESC            object

In [19]:



df.loc[df.PD_CD=='694','PD_DESC'] ='INCEST'

df.loc[df.PD_CD=='234','PD_DESC'] ='BURGLARY,UNKNOWN TIME'

internal = df[ ["PD_CD", "PD_DESC"] ].drop_duplicates().dropna()
internal['PD_CD'] = pd.Categorical(pd.to_numeric(internal['PD_CD'] ).astype(int))
internal = internal.set_index("PD_CD")
internal = internal.sort_index()
internal = internal.reset_index()
internal

,PD_CD,PD_DESC
0,100,STALKING COMMIT SEX OFFENSE
1,101,ASSAULT 3
2,102,ASSAULT SCHOOL SAFETY AGENT
3,103,ASSAULT TRAFFIC AGENT
4,104,VEHICULAR ASSAULT (INTOX DRIVE
...,...,...
427,916,LEAVING SCENE-ACCIDENT-PERSONA
428,918,RECKLESS DRIVING
429,922,"TRAFFIC,UNCLASSIFIED MISDEMEAN"
430,969,"TRAFFIC,UNCLASSIFIED INFRACTIO"


In [20]:
df.PD_CD.isnull().sum()

6271

In [21]:
# df = df[~df.PD_CD.isnull()]

In [22]:
df.PD_CD = pd.Categorical(df.PD_CD)

In [23]:
df = df.drop('PD_DESC', axis='columns')

### 11  CRM_ATPT_CPTD_CD   object

In [24]:
df.CRM_ATPT_CPTD_CD.value_counts()

COMPLETED    7244018
ATTEMPTED     125566
Name: CRM_ATPT_CPTD_CD, dtype: int64

In [25]:
df.CRM_ATPT_CPTD_CD = pd.Categorical(df.CRM_ATPT_CPTD_CD)

In [26]:
df.CRM_ATPT_CPTD_CD.isnull().sum()

7

In [27]:
# df = df [ ~df.CRM_ATPT_CPTD_CD.isnull() ]

### 12  LAW_CAT_CD         object

In [28]:
df.LAW_CAT_CD.value_counts()

MISDEMEANOR    4137277
FELONY         2273064
VIOLATION       959250
Name: LAW_CAT_CD, dtype: int64

In [29]:
df.LAW_CAT_CD = pd.Categorical(df.LAW_CAT_CD)

### 16  JURIS_DESC         object
### 17  JURISDICTION_CODE  object

In [30]:
df.JURISDICTION_CODE.isnull().sum()

6271

In [31]:
# df = df[ ~df.JURISDICTION_CODE.isnull() ]

jusridiction = df[ ["JURISDICTION_CODE", "JURIS_DESC", ] ].drop_duplicates().dropna()
jusridiction['JURISDICTION_CODE'] = pd.to_numeric(jusridiction['JURISDICTION_CODE'] )
jusridiction['JURISDICTION_CODE'] = jusridiction['JURISDICTION_CODE'].astype(int)
jusridiction = jusridiction.set_index("JURISDICTION_CODE")
jusridiction = jusridiction.sort_index()
jusridiction = jusridiction.reset_index()
jusridiction

,JURISDICTION_CODE,JURIS_DESC
0,0,N.Y. POLICE DEPT
1,1,N.Y. TRANSIT POLICE
2,2,N.Y. HOUSING POLICE
3,3,PORT AUTHORITY
4,4,TRI-BORO BRDG TUNNL
5,6,LONG ISLAND RAILRD
6,7,AMTRACK
7,8,CONRAIL
8,9,STATN IS RAPID TRANS
9,11,N.Y. STATE POLICE


In [32]:
df.JURISDICTION_CODE = pd.Categorical(df.JURISDICTION_CODE)


In [33]:
df = df.drop('JURIS_DESC', axis='columns')

###  13  BORO_NM            object

In [34]:
df.BORO_NM.value_counts()

BROOKLYN         2184784
MANHATTAN        1770127
BRONX            1598442
QUEENS           1462167
STATEN ISLAND     342750
Name: BORO_NM, dtype: int64

In [35]:
df.BORO_NM.isnull().sum()

11321

In [36]:
# df = df[~df.BORO_NM.isnull()]

In [37]:
df.BORO_NM = pd.Categorical(df.BORO_NM)

### 23  SUSP_AGE_GROUP     object
### 32  VIC_AGE_GROUP      object

In [38]:
# Both columns have a lot of noisy entries. We keep only the dominant groups, and also define an order
df.SUSP_AGE_GROUP = pd.Categorical(df.SUSP_AGE_GROUP, ordered=True, categories=['<18', '18-24',  '25-44', '45-64', '65+'])
df.VIC_AGE_GROUP = pd.Categorical(df.VIC_AGE_GROUP, ordered=True, categories=['<18', '18-24',  '25-44', '45-64', '65+'])


### 24  SUSP_RACE          object
### 25  SUSP_SEX           object

### 33  VIC_RACE           object
### 34  VIC_SEX            object

In [39]:
df.VIC_SEX.isnull().sum()

308

In [40]:
df.VIC_SEX.value_counts()

F    2885254
M    2432591
E    1150237
D     901197
U          4
Name: VIC_SEX, dtype: int64

In [41]:
# df = df[~df.VIC_SEX.isnull()]

df.VIC_SEX.replace(to_replace = 'U', value=np.nan, inplace = True)

In [42]:
df.VIC_RACE.isnull().sum()

309

In [43]:
df.VIC_RACE.value_counts()

UNKNOWN                           2431059
BLACK                             1768997
WHITE                             1267263
WHITE HISPANIC                    1190987
ASIAN / PACIFIC ISLANDER           426679
BLACK HISPANIC                     250862
AMERICAN INDIAN/ALASKAN NATIVE      33404
OTHER                                  31
Name: VIC_RACE, dtype: int64

In [44]:
df.VIC_RACE.replace(to_replace = 'OTHER', value='UNKNOWN', inplace = True)

In [45]:
# df = df[~df.VIC_RACE.isnull()]

In [46]:
df.SUSP_SEX.value_counts()

M    2392595
F     755649
U     663493
Name: SUSP_SEX, dtype: int64

In [47]:
# U is unknown, same is NULL.
df.SUSP_SEX.replace(to_replace = 'U', value=np.nan, inplace = True)

In [48]:
df.SUSP_RACE.value_counts()

BLACK                             1471671
UNKNOWN                           1032229
WHITE HISPANIC                     669200
WHITE                              430103
BLACK HISPANIC                     203530
ASIAN / PACIFIC ISLANDER           126348
AMERICAN INDIAN/ALASKAN NATIVE      11906
OTHER                                  11
Name: SUSP_RACE, dtype: int64

In [49]:
# Very small amount of OTHER values
df.SUSP_RACE.replace(to_replace = 'OTHER', value='UNKNOWN', inplace = True)



In [50]:
df.SUSP_RACE = pd.Categorical(df.SUSP_RACE)
df.SUSP_SEX = pd.Categorical(df.SUSP_SEX)
df.VIC_RACE = pd.Categorical(df.VIC_RACE)
df.VIC_SEX = pd.Categorical(df.VIC_SEX)

In [51]:
df.dtypes

CMPLNT_NUM                    int32
ADDR_PCT_CD                category
RPT_DT               datetime64[ns]
KY_CD                      category
PD_CD                      category
CRM_ATPT_CPTD_CD           category
LAW_CAT_CD                 category
BORO_NM                    category
LOC_OF_OCCUR_DESC            object
PREM_TYP_DESC                object
JURISDICTION_CODE          category
PARKS_NM                     object
HADEVELOPT                   object
HOUSING_PSA                  object
SUSP_AGE_GROUP             category
SUSP_RACE                  category
SUSP_SEX                   category
TRANSIT_DISTRICT             object
Latitude                     object
Longitude                    object
PATROL_BORO                  object
STATION_NAME                 object
VIC_AGE_GROUP              category
VIC_RACE                   category
VIC_SEX                    category
CMPLNT_FR            datetime64[ns]
CMPLNT_TO            datetime64[ns]
dtype: object

###  14  LOC_OF_OCCUR_DESC  object

In [52]:
df.LOC_OF_OCCUR_DESC.value_counts()

INSIDE         3744509
FRONT OF       1725874
OPPOSITE OF     195619
REAR OF         157175
OUTSIDE           3809
Name: LOC_OF_OCCUR_DESC, dtype: int64

In [53]:
df.LOC_OF_OCCUR_DESC.isnull().sum()

1542605

In [54]:
df.LOC_OF_OCCUR_DESC = pd.Categorical(df.LOC_OF_OCCUR_DESC)

### Latitude                     object
### Longitude                    object

In [55]:
df.Latitude = pd.to_numeric(df.Latitude, downcast='float')
df.Longitude  = pd.to_numeric(df.Longitude, downcast='float')

### TRANSIT_DISTRICT

In [56]:
df.TRANSIT_DISTRICT.value_counts()


4     27745
2     20207
3     15824
1     15777
20    14457
33    14366
12    12247
11    11276
32    11079
30    10499
34     7197
23     2692
Name: TRANSIT_DISTRICT, dtype: int64

In [57]:
len(df) - df.TRANSIT_DISTRICT.isnull().sum()

163366

In [58]:
df.drop('TRANSIT_DISTRICT', axis='columns', inplace=True)


### PREM_TYP_DESC

In [59]:
df.PREM_TYP_DESC.value_counts()

STREET                        2350379
RESIDENCE - APT. HOUSE        1563449
RESIDENCE-HOUSE                725297
RESIDENCE - PUBLIC HOUSING     552829
OTHER                          199601
                               ...   
CEMETERY                          854
MAILBOX INSIDE                    729
LOAN COMPANY                      494
TRAMWAY                           138
DAYCARE FACILITY                   53
Name: PREM_TYP_DESC, Length: 74, dtype: int64

In [60]:
df.PREM_TYP_DESC.isnull().sum()

40716

In [61]:
# df = df [~df.PREM_TYP_DESC.isnull()]

In [62]:
df.PREM_TYP_DESC = pd.Categorical(df.PREM_TYP_DESC)

In [63]:
df.PARKS_NM.value_counts()

CENTRAL PARK                               1641
FLUSHING MEADOWS CORONA PARK               1315
CONEY ISLAND BEACH & BOARDWALK             1059
WASHINGTON SQUARE PARK                      776
RIVERSIDE PARK                              614
                                           ... 
LOTT PARK                                     1
UNNAMED PARK ON JAY STREET                    1
THROGS NECK PARK                              1
UNNAMED PARK ON MAJOR DEEGAN EXPRESSWAY       1
UNNAMED PARK ON CLEARVIEW EXPRESSWAY          1
Name: PARKS_NM, Length: 1205, dtype: int64

In [64]:
df.PARKS_NM.value_counts().sum()

27620

In [65]:
df.drop('PARKS_NM', axis='columns', inplace=True)



 19  HADEVELOPT         object


In [66]:
df.HADEVELOPT.value_counts()

CASTLE HILL                 7546
VAN DYKE I                  6104
MARCY                       5595
BUTLER                      5226
GRANT                       5200
                            ... 
JEFFERSON                      1
344 EAST 28TH STREET           1
BRONXCHESTER                   1
FRANKLIN AVENUE III MHOP       1
1010 EAST 178TH STREET         1
Name: HADEVELOPT, Length: 279, dtype: int64

In [67]:
df.drop('HADEVELOPT', axis='columns', inplace=True)


 20  HOUSING_PSA        object



In [68]:
df.HOUSING_PSA.value_counts()

670      7320
887      7255
845      6773
720      6466
632      6320
         ... 
43662       1
41823       1
34685       1
9929        1
20264       1
Name: HOUSING_PSA, Length: 5101, dtype: int64

In [69]:
df.HOUSING_PSA.value_counts().sum()

566288

In [70]:
df.drop('HOUSING_PSA', axis='columns', inplace=True)

 30  PATROL_BORO        object


In [71]:
df.PATROL_BORO.value_counts()

PATROL BORO BRONX            1599224
PATROL BORO BKLYN SOUTH      1094785
PATROL BORO BKLYN NORTH      1091506
PATROL BORO MAN SOUTH         891047
PATROL BORO MAN NORTH         878274
PATROL BORO QUEENS NORTH      756395
PATROL BORO QUEENS SOUTH      708761
PATROL BORO STATEN ISLAND     342871
Name: PATROL_BORO, dtype: int64

In [72]:
df.PATROL_BORO = pd.Categorical(df.PATROL_BORO)

In [73]:
df.PATROL_BORO.isnull().sum()

6728

In [74]:
# df = df[~df.PATROL_BORO.isnull()] 

 31  STATION_NAME       object

In [75]:
df.STATION_NAME.value_counts()

125 STREET                        8199
14 STREET                         4522
34 ST.-PENN STATION               3845
42 ST.-PORT AUTHORITY BUS TERM    3731
116 STREET                        3353
                                  ... 
DISTRICT 12 OFFICE                  18
DISTRICT 30 OFFICE                  17
DISTRICT 34 OFFICE                  13
DISTRICT 23 OFFICE                   6
OFF-SYSTEM                           4
Name: STATION_NAME, Length: 372, dtype: int64

In [76]:
df.STATION_NAME.isnull().sum()

7206225

In [77]:
df.drop('STATION_NAME', axis='columns', inplace=True)

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7369591 entries, 0 to 7375992
Data columns (total 22 columns):
 #   Column             Dtype         
---  ------             -----         
 0   CMPLNT_NUM         int32         
 1   ADDR_PCT_CD        category      
 2   RPT_DT             datetime64[ns]
 3   KY_CD              category      
 4   PD_CD              category      
 5   CRM_ATPT_CPTD_CD   category      
 6   LAW_CAT_CD         category      
 7   BORO_NM            category      
 8   LOC_OF_OCCUR_DESC  category      
 9   PREM_TYP_DESC      category      
 10  JURISDICTION_CODE  category      
 11  SUSP_AGE_GROUP     category      
 12  SUSP_RACE          category      
 13  SUSP_SEX           category      
 14  Latitude           float32       
 15  Longitude          float32       
 16  PATROL_BORO        category      
 17  VIC_AGE_GROUP      category      
 18  VIC_RACE           category      
 19  VIC_SEX            category      
 20  CMPLNT_FR          datet

## Data exploration

In this part we check the different values that appear in the columns. When we detect noisy results, we delete the corresponding values. In fact, many of the operations that are performed above, in the 'data cleaning' section, are the result of observations that we make here. 

In [79]:
# Find the unique values in each column
# 
# df.describe(include = [np.object, 'category']).T['unique']
unique = df.describe(include = 'all').T['unique'].sort_values()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  after removing the cwd from sys.path.


In [80]:
unique

CRM_ATPT_CPTD_CD           2
SUSP_SEX                   2
LAW_CAT_CD                 3
VIC_SEX                    4
BORO_NM                    5
LOC_OF_OCCUR_DESC          5
VIC_AGE_GROUP              5
SUSP_AGE_GROUP             5
VIC_RACE                   7
SUSP_RACE                  7
PATROL_BORO                8
JURISDICTION_CODE         25
KY_CD                     74
PREM_TYP_DESC             74
ADDR_PCT_CD               77
PD_CD                    432
RPT_DT                  5479
CMPLNT_FR            2052354
CMPLNT_TO            2261791
CMPLNT_NUM               NaN
Latitude                 NaN
Longitude                NaN
Name: unique, dtype: object

In [81]:
for column in unique.index:
    if unique[column] < 200:
        print(df[column].value_counts())
        print("=====")

COMPLETED    7244018
ATTEMPTED     125566
Name: CRM_ATPT_CPTD_CD, dtype: int64
=====
M    2392595
F     755649
Name: SUSP_SEX, dtype: int64
=====
MISDEMEANOR    4137277
FELONY         2273064
VIOLATION       959250
Name: LAW_CAT_CD, dtype: int64
=====
F    2885254
M    2432591
E    1150237
D     901197
Name: VIC_SEX, dtype: int64
=====
BROOKLYN         2184784
MANHATTAN        1770127
BRONX            1598442
QUEENS           1462167
STATEN ISLAND     342750
Name: BORO_NM, dtype: int64
=====
INSIDE         3744509
FRONT OF       1725874
OPPOSITE OF     195619
REAR OF         157175
OUTSIDE           3809
Name: LOC_OF_OCCUR_DESC, dtype: int64
=====
25-44    2431632
45-64    1267207
18-24     748834
<18       338981
65+       259391
Name: VIC_AGE_GROUP, dtype: int64
=====
25-44    1015426
18-24     387578
45-64     361268
<18       108717
65+        30192
Name: SUSP_AGE_GROUP, dtype: int64
=====
UNKNOWN                           2431090
BLACK                             1768997
WHITE    

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7369591 entries, 0 to 7375992
Data columns (total 22 columns):
 #   Column             Dtype         
---  ------             -----         
 0   CMPLNT_NUM         int32         
 1   ADDR_PCT_CD        category      
 2   RPT_DT             datetime64[ns]
 3   KY_CD              category      
 4   PD_CD              category      
 5   CRM_ATPT_CPTD_CD   category      
 6   LAW_CAT_CD         category      
 7   BORO_NM            category      
 8   LOC_OF_OCCUR_DESC  category      
 9   PREM_TYP_DESC      category      
 10  JURISDICTION_CODE  category      
 11  SUSP_AGE_GROUP     category      
 12  SUSP_RACE          category      
 13  SUSP_SEX           category      
 14  Latitude           float32       
 15  Longitude          float32       
 16  PATROL_BORO        category      
 17  VIC_AGE_GROUP      category      
 18  VIC_RACE           category      
 19  VIC_SEX            category      
 20  CMPLNT_FR          datet

In [83]:
df['PREM_TYP_DESC'].value_counts().count()

74

In [84]:
# All columns, except for the dates and spatial coordinates, are categorical
# Columns with less than a few thousand unique values are good candidates 
# for ENUMs in the database given that the dataset is static.
# Also, in Pandas the internal representation becomes much more efficient
# as the Categoricals are stored as integers and not as strings
for column in unique.index:
    if column == 'RPT_DT':
        continue
    if df[column].value_counts().count() < 1000:
      df[column] = pd.Categorical(df[column])

In [85]:
# With all the proper data typing the dataset went down in size from 1.9Gb+ to 425Mb.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7369591 entries, 0 to 7375992
Data columns (total 22 columns):
 #   Column             Dtype         
---  ------             -----         
 0   CMPLNT_NUM         int32         
 1   ADDR_PCT_CD        category      
 2   RPT_DT             datetime64[ns]
 3   KY_CD              category      
 4   PD_CD              category      
 5   CRM_ATPT_CPTD_CD   category      
 6   LAW_CAT_CD         category      
 7   BORO_NM            category      
 8   LOC_OF_OCCUR_DESC  category      
 9   PREM_TYP_DESC      category      
 10  JURISDICTION_CODE  category      
 11  SUSP_AGE_GROUP     category      
 12  SUSP_RACE          category      
 13  SUSP_SEX           category      
 14  Latitude           float32       
 15  Longitude          float32       
 16  PATROL_BORO        category      
 17  VIC_AGE_GROUP      category      
 18  VIC_RACE           category      
 19  VIC_SEX            category      
 20  CMPLNT_FR          datet

In [86]:
df.memory_usage()

Index                58956728
CMPLNT_NUM           29478364
ADDR_PCT_CD           7372311
RPT_DT               58956728
KY_CD                 7372287
PD_CD                14759190
CRM_ATPT_CPTD_CD      7369715
LAW_CAT_CD            7369723
BORO_NM               7369803
LOC_OF_OCCUR_DESC     7369803
PREM_TYP_DESC         7372287
JURISDICTION_CODE     7370347
SUSP_AGE_GROUP        7369803
SUSP_RACE             7369947
SUSP_SEX              7369715
Latitude             29478364
Longitude            29478364
PATROL_BORO           7369955
VIC_AGE_GROUP         7369803
VIC_RACE              7369947
VIC_SEX               7369795
CMPLNT_FR            58956728
CMPLNT_TO            58956728
dtype: int64

In [87]:
df.dtypes

CMPLNT_NUM                    int32
ADDR_PCT_CD                category
RPT_DT               datetime64[ns]
KY_CD                      category
PD_CD                      category
CRM_ATPT_CPTD_CD           category
LAW_CAT_CD                 category
BORO_NM                    category
LOC_OF_OCCUR_DESC          category
PREM_TYP_DESC              category
JURISDICTION_CODE          category
SUSP_AGE_GROUP             category
SUSP_RACE                  category
SUSP_SEX                   category
Latitude                    float32
Longitude                   float32
PATROL_BORO                category
VIC_AGE_GROUP              category
VIC_RACE                   category
VIC_SEX                    category
CMPLNT_FR            datetime64[ns]
CMPLNT_TO            datetime64[ns]
dtype: object

In [88]:
df.dtypes

CMPLNT_NUM                    int32
ADDR_PCT_CD                category
RPT_DT               datetime64[ns]
KY_CD                      category
PD_CD                      category
CRM_ATPT_CPTD_CD           category
LAW_CAT_CD                 category
BORO_NM                    category
LOC_OF_OCCUR_DESC          category
PREM_TYP_DESC              category
JURISDICTION_CODE          category
SUSP_AGE_GROUP             category
SUSP_RACE                  category
SUSP_SEX                   category
Latitude                    float32
Longitude                   float32
PATROL_BORO                category
VIC_AGE_GROUP              category
VIC_RACE                   category
VIC_SEX                    category
CMPLNT_FR            datetime64[ns]
CMPLNT_TO            datetime64[ns]
dtype: object

## Storing in a MySQL database

In [89]:
!sudo pip3 install -U -q PyMySQL sqlalchemy sql_magic

     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 1.6 MB 31.2 MB/s 


In [90]:
import os
from sqlalchemy import create_engine

conn_string = 'mysql+pymysql://{user}:{password}@{host}/?charset=utf8mb4'.format(
    host = 'db.ipeirotis.org', 
    user = 'root',
    password = 'ae6jQniBS5muV27', #os.environ['MYSQL_PASSWORD'],
    encoding = 'utf8mb4')

engine = create_engine(conn_string)
con = engine.connect()

In [91]:
# Query to create a database
db_name = 'nypd'

sql = f"DROP DATABASE IF EXISTS {db_name}"
engine.execute(sql)

# Create a database
sql = f"CREATE DATABASE IF NOT EXISTS {db_name} DEFAULT CHARACTER SET 'utf8mb4'"
engine.execute(sql)


In [92]:

# And lets switch to the database
sql = f"USE {db_name}"
engine.execute(sql)


In [93]:
# In principle, we can let Pandas create the table, but we want to be a bit more predise
# with the data types, and we want to add documentation for each column
# from https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i


create_table_sql = '''
CREATE TABLE nypd (
  CMPLNT_NUM int,
  CMPLNT_FR datetime,
  CMPLNT_TO datetime,
  RPT_DT date,
  KY_CD SMALLINT,
  PD_CD SMALLINT,
  CRM_ATPT_CPTD_CD enum('COMPLETED','ATTEMPTED'),
  LAW_CAT_CD enum('FELONY','MISDEMEANOR','VIOLATION'),
  JURISDICTION_CODE SMALLINT,
  BORO_NM enum('BRONX','BROOKLYN','MANHATTAN','QUEENS','STATEN ISLAND'),
  ADDR_PCT_CD SMALLINT,
  LOC_OF_OCCUR_DESC enum('FRONT OF','INSIDE','OPPOSITE OF','OUTSIDE','REAR OF'),
  PATROL_BORO enum('PATROL BORO BRONX', 'PATROL BORO BKLYN SOUTH','PATROL BORO BKLYN NORTH','PATROL BORO MAN SOUTH','PATROL BORO MAN NORTH','PATROL BORO QUEENS NORTH','PATROL BORO QUEENS SOUTH','PATROL BORO STATEN ISLAND'),
  PREM_TYP_DESC varchar(30),
  SUSP_RACE enum('UNKNOWN', 'BLACK', 'WHITE', 'WHITE HISPANIC', 'ASIAN / PACIFIC ISLANDER', 'BLACK HISPANIC', 'AMERICAN INDIAN/ALASKAN NATIVE'),
  VIC_RACE enum('UNKNOWN', 'BLACK', 'WHITE', 'WHITE HISPANIC', 'ASIAN / PACIFIC ISLANDER', 'BLACK HISPANIC', 'AMERICAN INDIAN/ALASKAN NATIVE'),
  SUSP_AGE_GROUP enum('<18', '18-24',  '25-44', '45-64', '65+'),
  VIC_AGE_GROUP enum('<18', '18-24',  '25-44', '45-64', '65+'),
  SUSP_SEX enum('M', 'F'),
  VIC_SEX enum('M', 'F', 'E', 'D'),
  Latitude double,
  Longitude double,
  PRIMARY KEY (CMPLNT_NUM)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
'''
engine.execute(create_table_sql)

In [94]:
# Create a table
# See http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html for the documentation
from tqdm import tqdm
batchsize = 50000
batches = len(df) // batchsize + 1

t = tqdm(range(batches))

for i in t:
    # print("Batch:",i)
    # continue # Cannot execute this on Travis
    start = batchsize * i
    end = batchsize * (i+1)
    df[start:end].to_sql(
        name = 'nypd', 
        schema = db_name, 
        con = engine,
        if_exists = 'append',
        index = False, 
        chunksize = 1000)

100%|██████████| 148/148 [25:58<00:00, 10.53s/it]


In [95]:
engine.execute("CREATE INDEX ix_lat ON nypd.nypd(Latitude)")

In [96]:
engine.execute("CREATE INDEX ix_lon ON nypd.nypd(Longitude)")

In [97]:
engine.execute("CREATE INDEX ix_LAW_CAT_CD ON nypd.nypd(LAW_CAT_CD)")

In [98]:
engine.execute("CREATE INDEX ix_BORO_NM ON nypd.nypd(BORO_NM)")

In [99]:
engine.execute("CREATE INDEX ix_KY_CD ON nypd.nypd(KY_CD)")

In [100]:
engine.execute("CREATE INDEX ix_RPT_DT ON nypd.nypd(RPT_DT)")

In [101]:
engine.execute("CREATE INDEX ix_CMPLNT_FR ON nypd.nypd(CMPLNT_FR)")

In [102]:
engine.execute("DROP TABLE IF EXISTS offense_codes;")

create_table_sql = '''
CREATE TABLE offense_codes (
  KY_CD smallint,
  OFNS_DESC varchar(32),
  PRIMARY KEY (KY_CD)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
'''
engine.execute(create_table_sql)

offenses.to_sql(
        name = 'offense_codes', 
        schema = db_name, 
        con = engine,
        if_exists = 'append',
        index = False)

In [103]:
engine.execute("DROP TABLE IF EXISTS jurisdiction_codes;")

create_table_sql = '''
CREATE TABLE jurisdiction_codes (
  JURISDICTION_CODE smallint,
  JURIS_DESC varchar(40),
  PRIMARY KEY (JURISDICTION_CODE)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
'''
engine.execute(create_table_sql)


jusridiction.to_sql(
        name = 'jurisdiction_codes', 
        schema = db_name, 
        con = engine,
        if_exists = 'append',
        index = False)

In [104]:
internal.PD_DESC.str.len().max()


71

In [105]:
engine.execute("DROP TABLE IF EXISTS penal_codes;")

create_table_sql = '''
CREATE TABLE penal_codes (
  PD_CD smallint,
  PD_DESC varchar(80),
  PRIMARY KEY (PD_CD)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
'''
engine.execute(create_table_sql)


internal.to_sql(
        name = 'penal_codes', 
        schema = db_name, 
        con = engine,
        if_exists = 'append',
        index = False)

## TODO

### Add the penal code data as a separate table

`!curl 'https://data.cityofnewyork.us/api/views/qgea-i56i/files/65f25845-1551-4d21-91dc-869c977cd93d?download=true&filename=PDCode_PenalLaw.xlsx' -o PDCode_PenalLaw.xlsx`

### Examine whether to normalize 

The fields 

  
PREM_TYP_DESC    
HADEVELOPT    
PARKS_NM                     

would be better off as foreign keys or enums. They take too much space as strings.